## RMSE OVER 5 CLASSES

In [1]:
from data_Loader import dataLoaderPickle, justDatasetLoaderPickle
from networks_v2 import AttentionModel
from optimizer import train2, test2, optimizerNet
from optimizer import IterMeter

import torch
import torch.nn as nn
from comet_ml import Experiment

import matplotlib.pyplot as plt 
import numpy as np

import torch.nn.functional as F 
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split, SubsetRandomSampler, ConcatDataset

c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = justDatasetLoaderPickle('BALANCED_DATASET_ENG_SCRIPTED_FULL')

In [3]:
def train2(model, device, train_data, criterion, optimizer, scheduler, epochs, iter_meter, experiment):
    loss_list = []
    labels_list = []
    loss_tot_list = []

    for epoch in range(epochs):
        for x, y in train_data:
            optimizer.zero_grad()

            z = model(x.float())
            labels_list.extend(y.detach().numpy())
            z = F.log_softmax(z, dim=1)

            y = y.float()
            #z = z.squeeze(0)

            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            iter_meter.step()

            loss_list.append(loss.item())
            
        print('Train Epoch: {} \tLoss: {:.4f}\tRMSE: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.sqrt(np.mean(loss_list))
        ))
        loss_tot_list.append(np.mean(loss_list))
    
    #Printing training behaviour 
    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Training Loss')
    axs[0].set(xlabel= 'Epoch',ylabel='Loss')
    plt.show()

In [4]:
def test2(model, device, test_data, criterion, optimizer, scheduler, epochs, iter_meter, experiment):   
    loss_list = []
    labels_list = []
    loss_tot_list = []

    for epoch in range(epochs):
        for x, y in test_data:
            optimizer.zero_grad()
           
            z = model(x.float())
            labels_list.extend(y.detach().numpy())
            z = F.log_softmax(z, dim=0)

            y = y.float()
            z = z.squeeze(0)
            loss = criterion(z, y)

            loss.backward()
            optimizer.step()
            iter_meter.step()

            loss_list.append(loss.item())
            
        print('Test Epoch: {} \tLoss: {:.4f}\tRMSE: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.sqrt(np.mean(loss_list))
        ))
        loss_tot_list.append(np.mean(loss_list))

    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Test Loss')
    axs[0].set(xlabel= 'Epoch',ylabel='Loss')
    plt.show()

In [5]:
#Constants
learning_Rate = 0.0005
batch_size = 32
epochs = 10
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}
experiment = Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 6,
    "n_rnn_layers": 1,
    "rnn_dim": 256,
    "h_rnn_layers": 128,
    "n_class": 1,
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


In [6]:
#Building and training Model
model = AttentionModel(
    hparams['n_cnn_layers'],
    hparams['rnn_dim'],
    hparams['h_rnn_layers'],
    hparams['n_rnn_layers'],
    hparams['n_class'],
    hparams['stride'],
    hparams['dropout']
).to(device).float()

print(model)
print('Number Model Parameters', sum([param.nelement() for param in model.parameters()]))

#Optimizing Model
optimizer, scheduler = optimizerNet(model, hparams)

criterion = nn.MSELoss()
iter_meter = IterMeter()

AttentionModel(
  (dense): Conv1d(500, 32, kernel_size=(1,), stride=(1,))
  (cnn): Sequential(
    (0): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (2): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_runn

In [7]:
history = {'Train loss':[], 'Test Loss':[], 'Train Accuracy':[], 'Test Accuracy':[]}

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    print('Fold {}'.format(fold+1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

    train2(model, device, train_loader, criterion, optimizer, scheduler, epochs, iter_meter, experiment)
    test2(model, device, test_loader, criterion, optimizer, scheduler, epochs, iter_meter, experiment)

Fold 1


c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train Epoch: 0 	Loss: 5.8828	RMSE: 2.4254
Train Epoch: 1 	Loss: 5.8828	RMSE: 2.4255
Train Epoch: 2 	Loss: 5.8828	RMSE: 2.4254
Train Epoch: 3 	Loss: 5.8826	RMSE: 2.4254
Train Epoch: 4 	Loss: 5.8826	RMSE: 2.4254
Train Epoch: 5 	Loss: 5.8826	RMSE: 2.4254
Train Epoch: 6 	Loss: 5.8827	RMSE: 2.4254
Train Epoch: 7 	Loss: 5.8827	RMSE: 2.4254
Train Epoch: 8 	Loss: 5.8827	RMSE: 2.4254
Train Epoch: 9 	Loss: 5.8827	RMSE: 2.4254


c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([18])) that is different to the input size (torch.Size([18, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Epoch: 0 	Loss: 31.8332	RMSE: 5.6421
Test Epoch: 1 	Loss: 31.8485	RMSE: 5.6434
Test Epoch: 2 	Loss: 31.8679	RMSE: 5.6452
Test Epoch: 3 	Loss: 31.8667	RMSE: 5.6451
Test Epoch: 4 	Loss: 31.8763	RMSE: 5.6459
Test Epoch: 5 	Loss: 31.8787	RMSE: 5.6461
Test Epoch: 6 	Loss: 31.8855	RMSE: 5.6467
Test Epoch: 7 	Loss: 31.8803	RMSE: 5.6463
Test Epoch: 8 	Loss: 31.8790	RMSE: 5.6461
Test Epoch: 9 	Loss: 31.8786	RMSE: 5.6461
Fold 2
Train Epoch: 0 	Loss: 5.9006	RMSE: 2.4291
Train Epoch: 1 	Loss: 5.9005	RMSE: 2.4291
Train Epoch: 2 	Loss: 5.9008	RMSE: 2.4291
Train Epoch: 3 	Loss: 5.9009	RMSE: 2.4292
Train Epoch: 4 	Loss: 5.9009	RMSE: 2.4292
Train Epoch: 5 	Loss: 5.9008	RMSE: 2.4292
Train Epoch: 6 	Loss: 5.9008	RMSE: 2.4292
Train Epoch: 7 	Loss: 5.9008	RMSE: 2.4291
Train Epoch: 8 	Loss: 5.9008	RMSE: 2.4292
Train Epoch: 9 	Loss: 5.9007	RMSE: 2.4291
Test Epoch: 0 	Loss: 31.4808	RMSE: 5.6108
Test Epoch: 1 	Loss: 31.4876	RMSE: 5.6114
Test Epoch: 2 	Loss: 31.4792	RMSE: 5.6106
Test Epoch: 3 	Loss: 31.474

c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train Epoch: 0 	Loss: 5.9042	RMSE: 2.4299
Train Epoch: 1 	Loss: 5.9044	RMSE: 2.4299
Train Epoch: 2 	Loss: 5.9044	RMSE: 2.4299
Train Epoch: 3 	Loss: 5.9044	RMSE: 2.4299
Train Epoch: 4 	Loss: 5.9044	RMSE: 2.4299
Train Epoch: 5 	Loss: 5.9044	RMSE: 2.4299
Train Epoch: 6 	Loss: 5.9043	RMSE: 2.4299
Train Epoch: 7 	Loss: 5.9043	RMSE: 2.4299
Train Epoch: 8 	Loss: 5.9043	RMSE: 2.4299
Train Epoch: 9 	Loss: 5.9043	RMSE: 2.4299


c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Epoch: 0 	Loss: 31.2543	RMSE: 5.5906
Test Epoch: 1 	Loss: 31.2578	RMSE: 5.5909
Test Epoch: 2 	Loss: 31.2698	RMSE: 5.5919
Test Epoch: 3 	Loss: 31.2789	RMSE: 5.5928
Test Epoch: 4 	Loss: 31.2839	RMSE: 5.5932
Test Epoch: 5 	Loss: 31.2849	RMSE: 5.5933
Test Epoch: 6 	Loss: 31.2871	RMSE: 5.5935
Test Epoch: 7 	Loss: 31.2878	RMSE: 5.5935
Test Epoch: 8 	Loss: 31.2974	RMSE: 5.5944
Test Epoch: 9 	Loss: 31.2935	RMSE: 5.5941
Fold 7
Train Epoch: 0 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 1 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 2 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 3 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 4 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 5 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 6 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 7 	Loss: 5.8823	RMSE: 2.4253
Train Epoch: 8 	Loss: 5.8823	RMSE: 2.4254
Train Epoch: 9 	Loss: 5.8823	RMSE: 2.4253
Test Epoch: 0 	Loss: 31.8161	RMSE: 5.6406
Test Epoch: 1 	Loss: 31.7978	RMSE: 5.6390
Test Epoch: 2 	Loss: 31.7832	RMSE: 5.6377
Test Epoch: 3 	Loss: 31.789

KeyboardInterrupt: 